In [3]:
import pandas as pd
import json
import time
from google_currency import convert 


In [27]:
df=pd.read_csv('/Users/mikel/Documents/Projects/chameleon-pricing/country_currency.csv')
df.head()

,Country,last_value,Currency,Code
0,aruba,1.350690,Aruban Florin,AWG
1,afghanistan,17.914483,Afghani,AFN
2,angola,146.167168,Kwanza,AOA
3,albania,41.342391,Lek,ALL
4,united arab emirates,2.258503,UAE Dirham,AED


In [28]:
df[df.Country == 'germany']

,Country,last_value,Currency,Code
49,germany,0.744679,Euro,EUR


# PPP to dollar

https://pypi.org/project/google-currency/

!pip install google-currency

In [4]:
def ppp_to_dollar(Code, last_value):
        conversion=convert(Code, 'usd', last_value)
        result = json.loads(conversion)
        return result['amount']

In [150]:
ppp_to_dollar('SOS', 8229.53)

'14.07'

In [113]:
def ppp_spain_euros(ppp_to_dollar):
    conversion=convert('usd', 'eur', ppp_to_dollar)
    result = json.loads(conversion)
    return result['amount']

In [115]:
ppp_spain_euros(1)

'0.84'

In [133]:
def ppp_to_spain(ppp_to_dollar):
    return round(ppp_to_dollar*1.31, 2)

In [136]:
ppp_to_spain(0.76)

1.0

In [57]:
ppp_spain_euros(0.58)

'0.49'

In [98]:
%%time
apply = df.apply(
         lambda x: ppp_to_dollar(x['Code'], x['last_value']),
         axis=1)
df['ppp_todollar']  = apply
print(df)

In [137]:
%%time
apply = df.apply(
         lambda x: ppp_to_spain(x['ppp_todollar']),
         axis=1)
df['ppp_spain']  = apply

CPU times: user 2.39 ms, sys: 2.22 ms, total: 4.61 ms
Wall time: 8.32 ms


In [141]:
df[df.Country == 'spain']

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
50,spain,0.624484,Euro,EUR,0.76,1.0


In [104]:
df.dtypes

Country             object
last_value         float64
Currency            object
Code                object
ppp_todollar       float64
ppp_spain          float64
ppp_spain_euros     object
usd_eur            float64
dtype: object

In [15]:
df["ppp_todollar"] = df.ppp_todollar.astype(float)

In [143]:
df["ppp_spain"] = df.ppp_spain.astype(float)

In [144]:
df.dtypes

Country          object
last_value      float64
Currency         object
Code             object
ppp_todollar    float64
ppp_spain       float64
dtype: object

In [54]:
df[df.ppp_todollar == 0.00]

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain


In [18]:
df.drop(df[df['ppp_todollar'] == 0.0].index, inplace = True)

# Adjustment function

In [37]:
def price_calculator(product_price, country):
    find_ppp=df.ppp_spain[df.Country == country.lower()]
    return product_price * find_ppp

In [53]:
price_calculator(60, 'colombia')

36    29.4
Name: ppp_spain, dtype: float64

In [55]:
conversion=convert('usd', 'eur', 60)
result = json.loads(conversion)
result['amount']

'50.21'

In [67]:
df.to_excel('/Users/mikel/Documents/Projects/chameleon-pricing/DB.xlsx', index=False)

In [183]:
df=pd.read_excel('/Users/mikel/Documents/Projects/chameleon-pricing/DB.xlsx')
df

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
0,aruba,1.350690,Aruban Florin,AWG,0.75,0.98
1,afghanistan,17.914483,Afghani,AFN,0.23,0.30
2,angola,146.167168,Kwanza,AOA,0.23,0.30
3,albania,41.342391,Lek,ALL,0.41,0.54
4,united arab emirates,2.258503,UAE Dirham,AED,0.61,0.80
...,...,...,...,...,...,...
154,uruguay,25.351299,Peso Uruguayo,UYU,0.58,0.76
155,united states,1.000000,US Dollar,USD,1.00,1.31
156,uzbekistan,2085.548458,Uzbekistan Sum,UZS,0.20,0.26
157,south africa,6.652566,Rand,ZAR,0.48,0.63


# Get daily usd-eur exchange

In [91]:
def get_eur():
    conversion=convert('usd', 'eur', 1)
    result = json.loads(conversion)
    df['usd_eur']=result['amount']
    

In [75]:
d = {'usd': [1], 'eur': [0.86]}
usd_eur = pd.DataFrame(data=d)
usd_eur

,usd,eur
0,1,0.86


In [92]:
get_eur()

In [181]:
df.to_excel('/Users/mikel/Documents/Projects/chameleon-pricing/DB.xlsx', index=False)

In [182]:
df[df.ppp_todollar == 0]

,Country,last_value,Currency,Code,ppp_todollar,ppp_spain
